# Modificaciones ARIADNE

## Modelo

En `sed_fitting.py`, la función `model_grid` contiene el modelo que se ajusta. Por defecto es

$$ M = F \times \frac{R}{D}^2 $$

Que es justamente lo que [utiliza VOSA](http://svo2.cab.inta-csic.es/theory/vosa/helpw4.php?otype=star&action=help&what=fitbin) pero en el caso de una estrella. Es decir, hay que modificarlo a 

$$ M = F_1 \times \frac{R_1}{D}^2 + F_2 \times \frac{R_2}{D}^2 $$

Esto implica tener dos parámetros para el radio, además de duplicar los que necesite la función del flujo, para generar $F_2$

## Fit

Esta parte en principio no debería modificarse. Lo normal es que ARIADNE haga el BMA y todo lo que hace usualmente, pero por ahora no hay intenciones de cambiar eso. En el futuro sería deseable calcular la evidencia del modelo binario vs el single, si es que no hay una forma más eficiente de comparar modelos.

Lo que hace es obtener los residuos con `get_residuals`, que también devuelve los errores. Con esto se construye el likelihood via `log_likelihood`

Por tanto, lo que parece ser ideal es generar otra función `model_grid` al menos, que considere el caso binario. Luego queda ver si es necesario clonar las otras funciones para generar el likelihood o si se puede ser más directo.

## Pruebas

Para comenzar a ver si está funcionando bien y ser más eficientes en tiempo, partir por ajustar solo un parámetro y dejar los otros fijos. Verificar si se recupera el resultado correcto, comparar con caso single.

Ojalá recuperar los valores de la literatura para un sistema binario bien caracterizado, comparar qué tan cerca da, errores, etc.

## Dudas
ARIADNE permite dejar el radio fijo? Y otros valores como teff, logg, z? Me parece que si, hay que revisar la parte de los priors.

Podré ejecutar todo esto acá para hacer pruebas? Sin tener que modificar el codigo y reinstalar. Me parece que también sí, mientras pueda importar las funciones necesarias.

In [9]:
from extinction import apply
import numba as nb
import numpy as np

def get_interpolated_flux(temp, logg, z, filts, interpolators):
    """Interpolate the grid of fluxes in a given teff, logg and z.
    Parameters
    ----------
    temp: float
        The effective temperature.
    logg: float
        The superficial gravity.
    z: float
        The metallicity.
    filts: str
        The desired filter.
    Returns
    -------
    flux : float
        The interpolated flux at temp, logg, z for filter filt.
    """
    values = (logg, temp, z)
    flux = interpolators(values, filts)
    return flux

def get_residuals(theta, flux, flux_er, wave, filts, interpolators, use_norm,
                  av_law):
    """Calculate residuals of the model."""
    model = model_grid(theta, filts, wave, interpolators, use_norm, av_law)
    start = 5 if use_norm else 6
    inflation = theta[start:]
    residuals = flux - model
    errs = np.sqrt(flux_er ** 2 + inflation ** 2)
    return residuals, errs


def log_likelihood(theta, flux, flux_er, wave, filts, interpolators, use_norm,
                   av_law):
    """Calculate log likelihood of the model."""
    res, ers = get_residuals(theta, flux, flux_er, wave,
                             filts, interpolators, use_norm, av_law)

    lnl = fast_loglik(res, ers)

    if not np.isfinite(lnl):
        return -1e300

    return -.5 * lnl


@nb.njit
def fast_loglik(res, ers):
    ers2 = ers ** 2
    c = np.log(2 * np.pi * ers2)
    lnl = (c + (res ** 2 / ers2)).sum()
    return lnl

In [10]:
def binary_model_grid(theta, filts, wave, interpolators, use_norm, av_law):
    """Return the model grid in the selected filters.

    Parameters:
    -----------
    theta : array_like
        The parameters of the fit: teff, logg, z, radius, distance <-- aqui hay que cambiar
    star : Star
        The Star object containing all relevant information regarding the star. <-- esto no va, sino los filtros
    interpolators : dict
        A dictionary with the interpolated grid.
    use_norm : bool
        False for a full fit  (including radius and distance). True to fit
        for a normalization constant instead.

    Returns
    -------
    model : dict
        A dictionary whose keys are the filters and the values are the
        interpolated fluxes

    """
    Rv = 3.1  # For extinction.

    if use_norm:
        teff, logg, z, norm, Av = theta[:5]
    else:
        #Este es el caso de interes, hay que duplicar teff, logg, z y los radios
        #teff, logg, z, dist, rad, Av = theta[:6]
        
        teff1, logg1, z1, teff2, logg2, z2, dist, rad1, rad2, Av = theta[:10]
        dist *= 4.435e+7  # Transform from pc to solRad

    #Con esto se genera la SED (sin el coef R/D), deberia generar 2, uno por cada componente
    #flux = get_interpolated_flux(teff, logg, z, filts, interpolators)
    
    flux1 = get_interpolated_flux(teff1, logg1, z1, filts, interpolators)
    flux2 = get_interpolated_flux(teff2, logg2, z2, filts, interpolators)
    
    wav = wave * 1e4
    ext = av_law(wav, Av, Rv)
    if use_norm:
        model = apply(ext, flux) * norm
    else:
        #Este seria el modelo final, habria que agregar la otra componente
        #La parte de la extincion debiese ser la misma para ambos
        #model = apply(ext, flux) * (rad / dist) ** 2
        
        model = apply(ext, flux1)*(rad1/dist)**2 + apply(ext, flux2)*(rad2/dist)
    return model